# From poll to Index


## Setup

In [1]:
import os 
import sys 
import json 
import random
from matplotlib import pyplot
import pandas as pd
from pandas.plotting import scatter_matrix
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = "notebook"
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_PYTHON'] = sys.executable

In [2]:
spark = (
    SparkSession
        .builder
        .appName('poll_to_idxs')
        .getOrCreate()
    )

spark

## CSV loading and cleaning

In [3]:
#Directory where to access the poll
poll_dir = "../data/poll_data.csv"

# Load the csv poll as a dataframe
df = (
    spark.read.format("csv")
    # .option("encoding", "UTF-8")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(poll_dir)
)

# List of columns that will not be considered individually scored (due to not be used in the calculation)
skip_cols = [
    "Ho letto e accettato l'informativa e confermo inoltre di avere più di 18 anni",
    "Informazioni cronologiche",
    "Quanti anni hai?",
    "Genere",
    "Da quante persone è composto il tuo nucleo familiare?",
    "Occupazione",
    "Quanto è grande la tua azienda?",
    "Da che regione provieni?",
    "Provincia di provenienza",
    "In che regione lavori/studi?",
    "Provincia del luogo di lavoro/studio",
    "Invalidità",
    "Tipo di residenza",
    "Numero di persone con cui convivi",
    "Entrate Familiari Mensili Nette",
    "Entrate Personali Mensili Nette ",
]

# Load the questions json
with open("../data/questions.json", "r", encoding="utf-8") as questions_file:
    questions = json.load(questions_file)

# Create a question_text:question_idx map, useful to alias the columns
questions_idxs = {questions[q]["question_text"]: q for q in questions.keys()}

# Alias the columns (long column names cause bugs and are hard to use)
for k, v in questions_idxs.items():
    df = df.withColumnRenamed(k, v)

df.show(5)

+-------------------------+-----------------------------------------------------------------------------+-------------------+---------+-----------------------------------------------------+--------------------+-------------------------------+------------------------+------------------------+----------------------------+------------------------------------+------------------+--------------------+---------------------------------+-------------------------------+--------------------------------+---+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+----+--------------------+--------------------+--------------------+----+-------------------+--------------------+-------+--------------------+--------------------+---+---+------------------+--------------+---------------+----+--------------+------------+------------+--------------------+------------+----+--------------+-------------+---------------

## Computing the scores

In [4]:
# "Subtract" the list of unscorable cols from the list of scorable cols
scorable_cols = [item for item in df.columns if item not in skip_cols]

# Iterate over the scorable columns to create the scores dictionary
scores = {}
for col in scorable_cols:
    # Shortcuts for some useful values
    question_type = questions[col]["question_type"]
    question_score = questions[col]["question_score"]
    question_answers = questions[col]["answers"]

    # Skip unscored questions
    if len(question_answers) == 0 and question_type != "comma_separated":
        continue

    # Collect the Row values for the current column and access its value (index 0)
    answers = [str(d[0]) for d in df.select(col).collect()]

    # Handle score computation differently based on question_type
    row_scores = []
    if question_type in ["basic", "multivalue"]:
        for indexes_cols_df in answers:
            row_scores.append(question_answers[indexes_cols_df]["answer_score"] * question_score)

        scores[col] = row_scores

    elif question_type == "comma_separated":
        for i in answers:
            row_scores.append(len(i.split(";")))

        scores[col] = row_scores

In [5]:
# Create the scores dataframe from the scores dictionary
df_scores = spark.createDataFrame(pd.DataFrame(scores))
df_scores.show(5)

+----+----+-----+----+----+-----+----+----+-----+----+-----+-----+----+-----+---+---+-----+-----+-----+---+---+-----+-----+----+-----+-----+-----+----+----+-----+-----+-----+-----+----+-----+---+---+----+---+---+-----+---+---+----+---+----+----+-----+---+-----+----+----+---+-------+-------+-------+-------+-------+-------+-------+-------+-----+-----+----+
| S_1| S_2|  S_3| S_4| S_5|  S_6| S_7| S_8|  S_9|V_10| V_11| V_12|C_13| C_14|m_1|m_2|  m_3|  m_4|  m_5|m_6|m_7|  m_8|  m_9|m_10| m_11| m_12| m_13|m_14|m_15| m_16| m_17| m_18| m_19|m_20|  eh1|eh2|eh3| eh4|eh5|eh6|  ew1|ew2|ew3| ew4|ew5| ww1| ww2|  ww3|ww4|  wh1| wh2| wh3|wh4|wastew1|wastew2|wastew3|wastew4|wasteh1|wasteh2|wasteh3|wasteh4| ER_1| ER_2|ER_3|
+----+----+-----+----+----+-----+----+----+-----+----+-----+-----+----+-----+---+---+-----+-----+-----+---+---+-----+-----+----+-----+-----+-----+----+----+-----+-----+-----+-----+----+-----+---+---+----+---+---+-----+---+---+----+---+----+----+-----+---+-----+----+----+---+-------+---

In [6]:
# The indexes dictionary, which will be used to compute the indexes dataframe
indexes_cols = {}

## Computing the indexes

### Purchases indexes

In [7]:
# Purchases indexes calculation
df_s = df_scores.select([f"S_{d}" for d in range(1, 10)]).collect()

# i_S_col
i_S_col = []
for s in df_s:
    i_S_col.append(sum([val for val in s]) / 1450)
indexes_cols["i_S"] = i_S_col


col_S_1 = df.select("S_1").collect()
S_1 = []
answer1 = questions["S_1"]["answers"]
for i in range(len(col_S_1)):
    s1 = col_S_1[i][0]
    for j in answer1.keys():
        if s1 == j:
            val = answer1[j]["answer_score_1"]
            S_1.append(val)
            break

col_S_2 = df.select("S_2").collect()
S_2 = []
answer2 = questions["S_2"]["answers"]
for i in range(len(col_S_2)):
    s2 = str(col_S_2[i][0])
    for j in answer2.keys():
        if s2 == j:
            val = answer2[j]["answer_score_1"]
            S_2.append(val)
            break
        
df_s1 = df_scores.select([f"S_{d}" for d in range(3, 10)]).collect()
i_tot_S_col = []
k = 0  # variabile temporanea per accedere agli elementi di S_1 ed S_2 ed effettuare il rapporto
for s_1 in df_s1:
    somma = sum([val1 for val1 in s_1])
    r = S_2[k] / S_1[k]
    i_tot_S_col.append(somma * r)
    k += 1
indexes_cols["i_tot_S"] = i_tot_S_col

In [8]:
# Calcolo indici domande relative al Vestiario
df_v = df_scores.select([f"V_{d}" for d in range(10, 13)]).collect()

i_V_col = []
for v in df_v:
    i_V_col.append(sum([val for val in v]) / 550)

indexes_cols["i_V"] = i_V_col

col_V_10 = df.select("V_10").collect()
V_10 = []
answer10 = questions["V_10"]["answers"]
for i in range(len(col_V_10)):
    v10 = str(col_V_10[i][0])
    for j in answer10.keys():
        if v10 == j:
            val = answer10[j]["answer_score_1"]
            V_10.append(val)
            break
df_v1 = df_scores.select([f"V_{d}" for d in range(11, 13)]).collect()
i_tot_V_col = []
l = 0  # variabile temporanea per accedere agli elementi di V_10
for v1 in df_v1:
    somma1 = sum([val1 for val1 in v1]) / 400
    i_tot_V_col.append(somma1 * V_10[l])
    l += 1
indexes_cols["i_tot_V"] = i_tot_V_col

In [9]:
# Calcolo indici domande relative alla Casa
df_c = df_scores.select([f"C_{d}" for d in range(13, 15)]).collect()

i_C_col = []
for c in df_c:
    i_C_col.append(sum([val for val in c]) / 350)

indexes_cols["i_C"] = i_C_col

i_tot_C_col = []
for c1 in df_c:
    if c1[1] == 0:
        i_tot_C_col.append(c1[0] / 150)
    else:
        i_tot_C_col.append(c1[0] / c1[1])
indexes_cols["i_tot_C"] = i_tot_C_col

In [10]:
# Calcolo indice Educazione e Ricerca
df_e = df_scores.select([f"ER_{d}" for d in range(1, 4)]).collect()
i_ER_col = []
for e in df_e:
    i_ER_col.append(sum([val for val in e]) / 500)
indexes_cols["i_ER"] = i_ER_col

### Mobility indexes

In [11]:
df_m1 = df_scores.select(["m_8", "m_9"]).collect()

i_m1_col = []
for s in df_m1:
    m_8, m_9 = s[0], s[1]
    i_m1_col.append(m_9 / m_8)

indexes_cols["i_M1"] = i_m1_col

In [12]:
df_m2 = df_scores.select(["m_17", "m_18"]).collect()

i_m2_col = []
for s in df_m2:
    m_17, m_18 = s[0], s[1]
    i_m2_col.append(m_18 / m_17)

indexes_cols["i_M2"] = i_m2_col

In [13]:
fam_size_col_label = "Da quante persone è composto il tuo nucleo familiare?"

df_m3 = df_scores.select(["m_8"]).collect()
df_fam = df.select([fam_size_col_label]).collect()

i_m3_col = []
for s in df_m3:
    m_8, f = s[0], int(df_fam[0][0])
    i_m3_col.append(m_8 / f)

indexes_cols["i_M3"] = i_m3_col

In [14]:
df_m4 = df_scores.select(["m_11", "m_12", "m_13"]).collect()

i_m4_col = []
for s in df_m4:
    m_11, m_12, m_13 = s[0], s[1], s[2]
    i_m4_col.append(m_12 / (m_11 + m_12 + m_13))

indexes_cols["i_M4"] = i_m4_col

In [15]:
df_m5 = df_scores.select(["m_20"]).collect()

i_m5_col = []
for s in df_m5:
    m_20 = s[0]
    i_m5_col.append(m_20)

indexes_cols["i_M5"] = i_m5_col

In [16]:
df_m6 = df_scores.select(["m_19"]).collect()

i_m6_col = []
for s in df_m6:
    m_19 = s[0]
    i_m6_col.append(m_19)

indexes_cols["i_M6"] = i_m6_col

In [17]:
df_m7 = df_scores.select(["m_4", "m_5"]).collect()

i_m7_col = []
for s in df_m7:
    m_4, m_5 = s[0], s[1]
    i_m7_col.append(m_5 / m_4)

indexes_cols["i_M7"] = i_m7_col

### Energy indexes

In [18]:
# Systems (home)
df_e1 = df_scores.select(["eh2"]).collect()
i_e1_col = []
for s in df_e1:
    i_e1_col.append(s[0] / 5)
indexes_cols["i_e1"] = i_e1_col

In [19]:
# Systems (work)
df_e2 = df_scores.select(["ew2"]).collect()
i_e2_col = []
for s in df_e2:
    i_e2_col.append(s[0] / 5)
indexes_cols["i_e2"] = i_e2_col

In [20]:
# Sustainable Source (home)
df_e3 = df_scores.select(["eh3", "eh4"]).collect()
i_e3_col = []
for s in df_e3:
    eh3, eh4 = s[0], s[1]
    i_e3_col.append(eh3 * eh4)
indexes_cols["i_e3"] = i_e3_col

In [21]:
# Sustainable Source (work)
df_e4 = df_scores.select(["ew3", "ew4"]).collect()
i_e4_col = []
for s in df_e4:
    ew3, ew4 = s[0], s[1]
    i_e4_col.append(ew3 * ew4)
indexes_cols["i_e4"] = i_e4_col

In [22]:
# Efficiency Ratio
df_e5 = df_scores.select(["eh1", "eh5"]).collect()
i_e5_col = []
for s in df_e5:
    eh1, eh5 = s[0], s[1]
    i_e5_col.append(eh1 / eh5)
indexes_cols["i_e5"] = i_e5_col

In [23]:
# GreenBuilding (home)
df_e6 = df_scores.select(["eh6"]).collect()
i_e6_col = []
for s in df_e6:
    i_e6_col.append(s[0] / 6)
indexes_cols["i_e6"] = i_e6_col

In [24]:
# GreenBuilding (work)
df_e7 = df_scores.select(["ew5"]).collect()
i_e7_col = []
for s in df_e7:
    i_e7_col.append(s[0] / 6)
indexes_cols["i_e7"] = i_e7_col

### Water Index

In [25]:
# Water (home)
df_water1 = df_scores.select(["wh1", "wh2", "wh3", "wh4"]).collect()
i_e8_col = []
for s in df_water1:
    wh1, wh2, wh3, wh4 = s[0], s[1], s[2], s[3]
    i_e8_col.append((wh1 + wh2 + wh3 + wh4) / 4)
indexes_cols["i_e8"] = i_e8_col

In [26]:
# Water (work)
df_water2 = df_scores.select(["ww1", "ww2", "ww3", "ww4"]).collect()
i_e9_col = []
for s in df_water2:
    ww1, ww2, ww3, ww4 = s[0], s[1], s[2], s[3]
    i_e9_col.append((ww1 + ww2 + ww3 + ww4) / 4)
indexes_cols["i_e9"] = i_e9_col

### Waste Index

In [27]:
# Waste (home)
df_waste1 = df_scores.select(["wasteh1", "wasteh2", "wasteh3", "wasteh4"]).collect()
i_e10_col = []
for s in df_waste1:
    wasteh1, wasteh2, wasteh3, wasteh4 = s[0], s[1], s[2], s[3]
    i_e10_col.append((wasteh1 + wasteh2 + wasteh3 + wasteh4) / 4)
indexes_cols["i_e10"] = i_e10_col

In [28]:
# Waste (work)
df_waste2 = df_scores.select(["wastew1", "wastew2", "wastew3", "wastew4"]).collect()
i_e11_col = []
for s in df_waste1:
    wastew1, wastew2, wastew3, wastew4 = s[0], s[1], s[2], s[3]
    i_e11_col.append((wastew1 + wastew2 + wastew3 + wastew4) / 4)
indexes_cols["i_e11"] = i_e11_col

## Indexes dataframe

In [29]:
for k in indexes_cols.keys():
    indexes_cols[k] = [round(e, 2) for e in indexes_cols[k]]

In [30]:
indexes_cols_df = pd.DataFrame(indexes_cols)

indexes_max = {
    # "i_S": None,
    #"i_tot_S": None,
    # "i_V": None,
    # "i_C": None,
    # "i_ER": None,
    # "i_M1": 1,
    # "i_M3": None,
    # "i_M7": None,
    "i_tot_V": 5,
    "i_tot_C": 1.5,
    "i_M2": 1,
    "i_M4": 0.95,
    "i_M5": 100,
    "i_M6": 100,
    "i_e1": 1,
    "i_e2": 1,
    "i_e3": 5,
    "i_e4": 5,
    "i_e5": 1/0.015,
    "i_e6": 1,
    "i_e7": 1,
    "i_e8": 200,
    "i_e9": 200,
    "i_e10": 200,
    "i_e11": 200,
}

for c in indexes_cols_df.columns:
    val = indexes_cols_df[c] / indexes_max.get(c, indexes_cols_df[c].max())
    indexes_cols_df[c] = round(val, 3)

# create the indexes dataframe from the indexes dictionary
df_indexes = spark.createDataFrame(pd.DataFrame(indexes_cols_df))
df_indexes.show(52)

+-----+-------+-----+-------+----+-------+-----+-----+----+----+-----+----+----+-----+----+----+----+----+-----+----+----+-----+-----+-----+-----+
|  i_S|i_tot_S|  i_V|i_tot_V| i_C|i_tot_C| i_ER| i_M1|i_M2|i_M3| i_M4|i_M5|i_M6| i_M7|i_e1|i_e2|i_e3|i_e4| i_e5|i_e6|i_e7| i_e8| i_e9|i_e10|i_e11|
+-----+-------+-----+-------+----+-------+-----+-----+----+----+-----+----+----+-----+----+----+----+----+-----+----+----+-----+-----+-----+-----+
|0.623|  0.333|0.634|   0.25|0.28|   0.66|0.389|0.796| 1.0| 0.6|  0.4| 0.0| 0.0|0.665| 0.2| 0.2|0.05|0.05| 0.25|0.17|0.17|0.188| 0.25|0.312|0.312|
| 0.58|  0.303|0.415|   0.15|0.14|    0.0|0.389|0.359| 0.2| 1.0|0.347| 0.5| 1.0|  0.5| 0.2| 0.4| 0.0| 0.2|  1.0|0.33|0.17|  0.0|0.188|  1.0|  1.0|
|0.536|  0.607| 0.22|   0.05|0.29|    0.0|  0.0| 0.12|0.33| 1.0|0.084| 0.0| 0.0| 0.05| 0.2| 0.2| 0.0| 0.0|0.167|0.17|0.17|0.125|  0.0|  0.5|  0.5|
|0.478|  0.137|0.195|   0.05|0.29|    0.0|0.444|  1.0| 1.0| 0.6|0.021| 0.5| 0.0|0.665| 0.2| 0.2| 0.0| 0.2|  1.0|0.33|0

In [31]:
df_features = df_indexes.toPandas()

vectors = []
for _, r in df_features.iterrows():
    values = []
    for i in range(len(df_features.columns)):
        values.append(r[i])

    vectors.append(Vectors.dense(values))

In [32]:
df_vectors = spark.createDataFrame(pd.DataFrame({"features":vectors}))

## Collaborative Filtering

In [33]:
user_id = [i for i in range(indexes_cols_df.shape[0])]

rows = []
for i in user_id:
    for index,c in enumerate(indexes_cols_df):
        rows.append((i, index, indexes_cols_df.iloc[i][index]))

cf_df = pd.DataFrame(rows, columns=["user_id", "feature_id", "value"])


cf_df = spark.createDataFrame(cf_df)

In [34]:
ratings = cf_df
(training, test) = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(
    maxIter=5, 
    regParam=0.01, 
    userCol="user_id", 
    itemCol="feature_id", 
    ratingCol="value",
    coldStartStrategy="drop",
)

model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(
    metricName="rmse", 
    labelCol="value",
    predictionCol="prediction",
)

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)

Root-mean-square error = 0.2517050357200105


In [35]:
punctured_df = indexes_cols_df
celle_el = []
c = 1
while c <= 130:
    i = random.randint(0,51)
    j = random.randint(0,24)
    cel = (i,j)
    if cel in celle_el:
        continue
    else:
        punctured_df.iloc[i,j] = None
        celle_el.append(cel)
        c += 1
print(celle_el)
print(len(celle_el))
punctured_df

[(0, 8), (18, 24), (19, 3), (35, 19), (6, 15), (26, 19), (30, 10), (40, 9), (1, 13), (2, 10), (7, 11), (19, 12), (16, 16), (48, 3), (45, 3), (24, 22), (8, 1), (42, 18), (7, 22), (37, 15), (11, 21), (32, 1), (20, 1), (38, 8), (0, 18), (20, 23), (46, 2), (33, 17), (41, 11), (26, 17), (39, 1), (48, 6), (35, 17), (40, 14), (5, 3), (20, 12), (18, 5), (41, 13), (47, 19), (37, 11), (11, 12), (30, 15), (4, 1), (44, 5), (26, 16), (4, 24), (9, 11), (32, 24), (48, 24), (14, 11), (18, 17), (25, 1), (27, 5), (28, 12), (21, 0), (49, 23), (12, 20), (50, 16), (23, 12), (37, 9), (37, 7), (50, 4), (23, 5), (35, 23), (14, 2), (36, 6), (28, 20), (23, 14), (9, 24), (34, 13), (9, 19), (21, 22), (26, 5), (27, 15), (11, 6), (47, 4), (49, 3), (4, 11), (4, 21), (49, 5), (48, 23), (12, 16), (43, 22), (47, 24), (18, 1), (49, 24), (30, 19), (13, 14), (32, 12), (23, 2), (9, 15), (49, 22), (19, 1), (42, 24), (6, 3), (8, 15), (46, 4), (7, 6), (38, 14), (14, 14), (21, 20), (33, 0), (18, 18), (31, 5), (22, 22), (39, 4)

,i_S,i_tot_S,i_V,i_tot_V,i_C,i_tot_C,i_ER,i_M1,i_M2,i_M3,...,i_e2,i_e3,i_e4,i_e5,i_e6,i_e7,i_e8,i_e9,i_e10,i_e11
0,0.623,0.333,0.634,0.250,0.28,0.660,0.389,0.796,NaN,0.6,...,0.2,0.05,0.05,NaN,0.17,0.17,0.188,0.250,0.312,0.312
1,0.580,0.303,0.415,0.150,0.14,0.000,0.389,0.359,0.20,1.0,...,0.4,0.00,0.20,1.000,0.33,0.17,0.000,0.188,1.000,1.000
2,0.536,0.607,0.220,0.050,0.29,0.000,0.000,0.120,0.33,1.0,...,0.2,0.00,0.00,0.167,0.17,0.17,0.125,0.000,0.500,0.500
3,0.478,0.137,0.195,0.050,0.29,0.000,0.444,1.000,1.00,0.6,...,0.2,0.00,0.20,1.000,0.33,0.17,0.000,0.000,1.000,1.000
4,0.464,NaN,0.000,0.000,NaN,0.000,0.000,0.240,1.00,1.0,...,0.2,0.00,0.00,1.000,0.17,0.17,NaN,0.000,0.438,NaN
5,0.565,0.181,0.000,NaN,0.43,0.000,0.000,0.120,0.20,1.0,...,0.2,0.10,0.10,0.000,0.17,0.33,0.000,0.000,0.000,0.000
6,0.855,0.683,0.524,NaN,0.43,0.000,0.389,0.120,0.20,1.0,...,NaN,0.00,0.00,0.250,0.17,0.33,0.250,0.000,NaN,1.000
7,0.406,0.076,0.110,0.024,0.43,0.327,NaN,0.198,0.33,0.6,...,0.2,0.00,0.00,0.500,0.33,0.33,0.000,NaN,0.000,0.000
8,0.667,NaN,0.439,0.100,0.57,0.220,0.000,0.120,0.20,1.0,...,NaN,0.00,0.05,0.000,0.33,0.17,0.062,0.000,0.500,0.500
9,0.580,0.667,0.671,0.150,0.71,0.440,0.000,0.120,0.50,1.0,...,NaN,0.15,0.05,0.750,NaN,0.33,0.125,0.125,0.750,NaN


In [36]:
user_id = [i for i in range(punctured_df.shape[0])]

rows = []
for i in user_id:
    for index,c in enumerate(indexes_cols_df):
        rows.append((i, index, punctured_df.iloc[i][index]))

punctured_cf_df = pd.DataFrame(rows, columns=["user_id", "feature_id", "value"])


punctured_cf_df = spark.createDataFrame(punctured_cf_df)
punctured_cf_df.show(1300)

+-------+----------+-----+
|user_id|feature_id|value|
+-------+----------+-----+
|      0|         0|0.623|
|      0|         1|0.333|
|      0|         2|0.634|
|      0|         3| 0.25|
|      0|         4| 0.28|
|      0|         5| 0.66|
|      0|         6|0.389|
|      0|         7|0.796|
|      0|         8|  NaN|
|      0|         9|  0.6|
|      0|        10|  0.4|
|      0|        11|  0.0|
|      0|        12|  0.0|
|      0|        13|0.665|
|      0|        14|  0.2|
|      0|        15|  0.2|
|      0|        16| 0.05|
|      0|        17| 0.05|
|      0|        18|  NaN|
|      0|        19| 0.17|
|      0|        20| 0.17|
|      0|        21|0.188|
|      0|        22| 0.25|
|      0|        23|0.312|
|      0|        24|0.312|
|      1|         0| 0.58|
|      1|         1|0.303|
|      1|         2|0.415|
|      1|         3| 0.15|
|      1|         4| 0.14|
|      1|         5|  0.0|
|      1|         6|0.389|
|      1|         7|0.359|
|      1|         8|  0.2|
|

In [38]:
ratings = punctured_cf_df
(training, test) = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(
    maxIter=5, 
    regParam=0.01, 
    userCol="user_id", 
    itemCol="feature_id", 
    ratingCol="value",
    coldStartStrategy="drop",
)

model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(
    metricName="rmse", 
    labelCol="value",
    predictionCol="prediction",
)

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)

Py4JJavaError: An error occurred while calling o903.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 177.0 failed 1 times, most recent failure: Lost task 6.0 in stage 177.0 (TID 500) (LAPTOP-FRANCESCA.station executor driver): java.lang.RuntimeException: Ratings MUST NOT be Null or NaN
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.project_doConsume_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.

## K-means

In [ ]:
kmeans = KMeans(k=3)

kmeans.setSeed(1)
# kmeans.setWeightCol("weigh_col")
kmeans.setMaxIter(10)
kmeans.getMaxIter()
kmeans.clear(kmeans.maxIter)
kmeans.getSolver()

model = kmeans.fit(df_vectors)

model.getMaxBlockSizeInMB()
model.getDistanceMeasure()
model.setPredictionCol("prediction")
model.predict(df_vectors.head().features)

centers = model.clusterCenters()
len(centers)

transformed = model.transform(df_vectors).select("features", "prediction")
rows = transformed.collect()

## Plots

In [ ]:
sns.heatmap(df_features.corr(), cmap="crest")

In [ ]:
with pd.option_context('display.max_rows', 100, 'display.max_columns', 100):
    display(df_features.corr()[df_features.corr() >= 0.5])

In [ ]:
# df_features.plot.scatter('i_S', 'i_tot_S', alpha=0.2)
# df_features.plot.scatter('i_C', 'i_tot_C', alpha=0.2)
# df_features.plot.scatter('i_V', 'i_tot_V', alpha=0.2)
# df_features.plot.scatter('i_e8', 'i_e3', alpha=0.2)
# df_features.plot.scatter('i_e7', 'i_e6', alpha=0.2)
# df_features.plot.scatter('i_e9', 'i_e8', alpha=0.2)

#    casa  i_e_1, i_e_3, i_e_6
#  lavoro  i_e_2, i_e_4, i_e_7


scatter_matrix(df_features[['i_e1', 'i_e3', 'i_e6', 'i_e2', 'i_e4', 'i_e7']], diagonal='kde')
scatter_matrix(df_features[['i_e1', 'i_e3', 'i_e6', 'i_e2', 'i_e4', 'i_e7']], diagonal='hist')

scatter_matrix(df_features[['i_e8', 'i_e9', 'i_e10', 'i_e11']], diagonal='kde')
scatter_matrix(df_features[['i_e8', 'i_e9', 'i_e10', 'i_e11']], diagonal='hist')

scatter_matrix(df_features[['i_S', 'i_tot_S', 'i_V', 'i_tot_V', 'i_C', 'i_tot_C']], diagonal='kde')
scatter_matrix(df_features[['i_S', 'i_tot_S', 'i_V', 'i_tot_V', 'i_C', 'i_tot_C']], diagonal='hist')

In [ ]:
fig, ax = pyplot.subplots()
pyplot.xlabel('casa')
pyplot.ylabel('lavoro')
ax.scatter(df_features['i_e8'], df_features['i_e9'])

In [ ]:
print(model.hasSummary)
summary = model.summary
print(summary.k)
print(summary.clusterSizes)

summary.cluster.show(52)